In [1]:
import os
import pandas as pd
import FinanceDataReader as fdr
input_path = "C:/Users/Gilseung/Downloads/기러기"
output_path = "C:/Users/Gilseung/Desktop/Jupyter/GILLAB/QUANT_DATA/201609~202108/재무제표"

In [2]:
def parsing_cash_flow(file_name):
    f = open(input_path + "/" + file_name)
    line = f.readline() # 한 줄 읽기

    investment = dict() # 투자활동현금흐름 
    finance = dict() # 재무활동현금흐름
    sales = dict() # 영업활동현금흐름 

    while line: # 읽을게 있을 때까지 반복
        for word in ["투자활동현금흐름", "재무활동현금흐름", "영업활동현금흐름"]:
            if word in line: # 영업활동현금흐름이 있으면, 
                s_line = line.split("\t") # 탭을 기준으로 분할
                code = s_line[1][1:-1] # 코드는 1번째 위치에 있고 [00660]처럼 되어 있음
                name = s_line[2] # 회사명은 2번째 위치에 있음
                try:
                    idx = s_line.index(word) # 영업활동현금흐름이 있는 위치를 찾고
                except:
                    for i, temp in enumerate(s_line):
                        if word in temp: # I. 영업활동현금흐름처럼 되어 있는 경우도 있어서 예외 처리
                            idx = i 
                            break
                try:
                    value = s_line[idx+1] # 바로 뒤에 있는 값을 사용 
                    value = int(value.replace(',', '')) # 콤마가 있는 경우가 있어서 콤마를 삭제하고 int로 변경
                except:
                    value = 0 # 값이 비어있는 경우에는 0으로 넣기

                if word == "투자활동현금흐름":
                    investment[(code, name)] = value # (code, name)을 키로하여 value를 추가
                elif word == "재무활동현금흐름":
                    finance[(code, name)] = value # (code, name)을 키로하여 value를 추가
                else:
                    sales[(code, name)] = value # (code, name)을 키로하여 value를 추가

        line = f.readline() # 다음 줄 읽기
    return investment, finance, sales

In [3]:
# 값이 없는 것을 NaN으로 채우기 위해 가져옴
stock_list = pd.concat([fdr.StockListing("KOSPI"), fdr.StockListing("KOSDAQ")]).dropna()
stock_list = stock_list[['Symbol', 'Name']]
stock_list.columns = ["종목코드", "종목명"]

In [4]:
# 2016년 1분기로 먼저 수행 (merge를 쉽게하기 위함)
investment, finance, sales = parsing_cash_flow("2016_1분기보고서_04_현금흐름표_연결_20210317.txt")

investment_df = pd.Series(investment).reset_index()
investment_df.columns = ["종목코드", "종목명","2016-1Q"]
investment_df = pd.merge(stock_list, investment_df, on = ["종목코드", "종목명"], how = "left") 

finance_df = pd.Series(finance).reset_index()
finance_df.columns = ["종목코드", "종목명","2016-1Q"]
finance_df = pd.merge(stock_list, finance_df, on = ["종목코드", "종목명"], how = "left")

sales_df = pd.Series(sales).reset_index()
sales_df.columns = ["종목코드", "종목명","2016-1Q"]
sales_df = pd.merge(stock_list, sales_df, on = ["종목코드", "종목명"], how = "left")

In [5]:
for year in [2016, 2017, 2018, 2019, 2020, 2021]:
    for quarter_name, quarter in zip(["1분기", "반기", "3분기", "사업"], ["1Q", "2Q", "3Q", "4Q"]):
        if year == 2016 and quarter == "1Q": # 2016 1분기는 이미 했으므로 패스
            continue
        
        for file in os.listdir(input_path): 
            if ("연결" in file) and (str(year) + "_" in file) and (quarter_name in file):
                # 조건에 만족하는 파일을 찾으면 해당 파일을 열어서 정리
                investment, finance, sales = parsing_cash_flow(file)
                
                investment = pd.Series(investment).reset_index()
                investment.columns = ["종목코드", "종목명","{}-{}".format(year, quarter)]
                investment = pd.merge(stock_list, investment, on = ["종목코드", "종목명"], how = "left") 
                investment_df = pd.merge(investment_df, investment, on = ["종목코드", "종목명"], how = "outer") # 데이터 병합
                
                finance = pd.Series(finance).reset_index()
                finance.columns = ["종목코드", "종목명","{}-{}".format(year, quarter)]
                finance = pd.merge(stock_list, finance, on = ["종목코드", "종목명"], how = "left") 
                finance_df = pd.merge(finance_df, finance, on = ["종목코드", "종목명"], how = "outer")

                sales = pd.Series(sales).reset_index()
                sales.columns = ["종목코드", "종목명","{}-{}".format(year, quarter)]
                sales = pd.merge(stock_list, sales, on = ["종목코드", "종목명"], how = "left")
                sales_df = pd.merge(sales_df, sales, on = ["종목코드", "종목명"], how = "outer")

In [7]:
investment_df.to_csv(output_path + "/" + "투자활동현금흐름.csv", index = False, encoding = "cp949")
finance_df.to_csv(output_path + "/" + "재무활동현금흐름.csv", index = False, encoding = "cp949")
sales_df.to_csv(output_path + "/" + "영업활동현금흐름.csv", index = False, encoding = "cp949")

In [8]:
investment_df

,종목코드,종목명,2016-1Q,2016-2Q,2016-3Q,2016-4Q,2017-1Q,2017-2Q,2017-3Q,2017-4Q,...,2019-1Q,2019-2Q,2019-3Q,2019-4Q,2020-1Q,2020-2Q,2020-3Q,2020-4Q,2021-1Q,2021-2Q
0,095570,AJ네트웍스,-7.211772e+09,-4.532520e+10,-5.735398e+10,-8.969971e+10,-2.388593e+10,-4.371911e+10,-6.473486e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,006840,AK홀딩스,-1.144177e+10,-1.134679e+11,-1.346598e+11,-1.285956e+11,-9.730340e+10,-1.626234e+11,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,027410,BGF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.508857e+11,...,-3.049230e+09,1.395209e+10,1.317579e+10,-4.021998e+08,-5.704160e+09,1.913969e+09,-3.057699e+10,-2.923150e+10,6.418713e+09,-3.684233e+08
3,282330,BGF리테일,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.210338e+10,...,-3.033502e+10,-1.143832e+11,-1.812811e+11,-2.939092e+11,-2.817577e+09,-9.743297e+10,-1.982992e+11,-2.663609e+11,-2.178408e+10,-1.801935e+11
4,138930,BNK금융지주,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2150,145020,휴젤,NaN,NaN,NaN,NaN,NaN,NaN,-4.555215e+11,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2151,010240,흥국,-5.389748e+07,-1.150318e+08,-7.188761e+07,-2.458423e+08,-2.381453e+08,1.114661e+09,8.874041e+08,4.920951e+08,...,-1.968295e+07,-9.207460e+08,-1.831941e+09,-3.905895e+09,-5.205148e+09,-4.153398e+09,-3.431445e+09,-4.112492e+09,-3.093279e+09,-3.517915e+09
2152,189980,흥국에프엔비,1.632418e+09,-2.418498e+08,-1.621331e+09,-4.234399e+08,-2.242555e+09,-4.133552e+09,-4.316002e+09,-6.842367e+09,...,-6.836456e+08,-3.583807e+09,7.821592e+08,-2.269475e+09,-4.594065e+09,-6.358355e+09,-8.691481e+09,-4.720759e+09,2.342602e+08,-1.690976e+09
2153,037440,희림,7.775837e+08,7.085425e+08,-3.464380e+07,-5.451050e+08,-1.564911e+09,-1.141548e+09,-1.832678e+09,-1.619932e+09,...,-7.384685e+08,-1.002737e+09,-1.762948e+09,-6.207095e+09,-5.032305e+09,-5.053706e+09,-6.256682e+09,-5.588194e+09,-2.155295e+09,-1.269377e+09
